# SPCS PostgreSQL Database Connectivity Test

This notebook provides modular components for testing PostgreSQL connectivity from Snowflake Container Services (SPCS):

## Quick Start Options:
1. **Basic Test**: Configure parameters and run network/auth tests
2. **Full Setup**: Use cells to set up PyPI access and PostgreSQL EAI as needed
3. **Custom Setup**: Pick and choose individual components as needed

## Components Available:
- **Configuration**: Set your PostgreSQL connection parameters
- **PyPI Setup**: Optional - enables installing pg8000 driver from PyPI
- **Network Test**: Tests basic network connectivity to PostgreSQL host
- **Authentication Test**: Tests PostgreSQL authentication and basic queries using pg8000
- **PostgreSQL EAI**: Optional - configures External Access Integration for PostgreSQL


**Note: You must restart the Session after making EAI changes for them to take effect**

In [ ]:
# PostgreSQL Connectivity Test Configuration
# Update these parameters with your actual PostgreSQL connection details

# User Configuration - UPDATE THIS
POSTGRES_HOST = "aurora-postgresql.cluster-abcdef122.us-east-2.rds.amazonaws.com"
POSTGRES_PORT = 5432
POSTGRES_DATABASE = "postgres"
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "password"

print(f"Configuration:")
print(f"PostgreSQL Host: {POSTGRES_HOST}")
print(f"PostgreSQL Port: {POSTGRES_PORT}")
print(f"PostgreSQL Database: {POSTGRES_DATABASE}")
print(f"PostgreSQL User: {POSTGRES_USER}")
print("\nReady to test connectivity...")

## 1. PyPI Setup (Optional)

Run these cells if you need to install the pg8000 PostgreSQL driver from PyPI. This creates the necessary network rules and External Access Integration for PyPI access.

**Skip this section if you already have pg8000 installed or have PyPI access configured.**


In [ ]:
-- Create Network Rule and External Access Integration for PyPI
-- Run this cell to enable installing Python packages from PyPI

CREATE OR REPLACE NETWORK RULE pypi_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('pypi.org', 'pypi.python.org', 'pythonhosted.org', 'files.pythonhosted.org');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION pypi_access_integration
  ALLOWED_NETWORK_RULES = (pypi_network_rule)
  ENABLED = true
  COMMENT = 'External Access Integration for PyPI package installation';

-- Grant usage on the integration
GRANT USAGE ON INTEGRATION pypi_access_integration TO ROLE ACCOUNTADMIN;

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'pypi_access_integration';


In [ ]:
-- Apply PyPI integration to this notebook
-- Run this after creating the PyPI integration above

ALTER NOTEBOOK EAI_POSTGRES
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('pypi_access_integration');


**NOTE: You will need to restart the Notebook session after applying an EAI**

In [ ]:
# Install PostgreSQL driver (pg8000)
# Make sure PyPI access is configured first if you get connection errors
# You can run this cell twice; the first to install the driver, the second to confirm it is imported

try:
    import pg8000
    print("✅ pg8000 already available")
except ImportError:
    print("📦 Installing pg8000...")
    %pip install pg8000
    print("✅ pg8000 installed")

## 2. Connectivity Tests

These cells test connectivity to your PostgreSQL database. Run them in order to diagnose any connection issues.

**Note: If you need to install pg8000, make sure to run the PyPI setup section first and restart your session.**


In [ ]:
# Network Connectivity Test
# Tests basic network connectivity to PostgreSQL host and port

import socket

def test_network_connectivity(host, port, description):
    """Test network connectivity to PostgreSQL host and port"""
    try:
        print(f"🔍 Testing {description}: {host}:{port}")

        # Test network connectivity
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(10)
        result = sock.connect_ex((host, port))
        sock.close()

        if result == 0:
            print(f"  ✅ Network connection successful")
            return True
        else:
            print(f"  ❌ Network connection failed (error code: {result})")
            print(f"  💡 This may indicate the need for a PostgreSQL EAI")
            return False

    except Exception as e:
        print(f"  ❌ Network Error: {e}")
        return False

# Run network connectivity test
print("=" * 50)
print("NETWORK CONNECTIVITY TEST")
print("=" * 50)

network_result = test_network_connectivity(POSTGRES_HOST, POSTGRES_PORT, "PostgreSQL Network Connectivity")

if network_result:
    print(f"\n✅ Network connectivity PASSED - PostgreSQL host is reachable")
    print("You can proceed to test PostgreSQL authentication.")
else:
    print(f"\n❌ Network connectivity FAILED")
    print("You need to configure a PostgreSQL External Access Integration (EAI).")
    print("See the PostgreSQL EAI Setup section below.")

In [ ]:
# PostgreSQL Authentication Test
# Tests PostgreSQL authentication and basic database operations
# Run this after confirming network connectivity works

def test_postgres_authentication():
    """Test PostgreSQL authentication and basic query"""
    try:
        print(f"🔍 Testing PostgreSQL Authentication and Basic Query")

        # Import pg8000 library
        try:
            import pg8000
            print("  📦 Using pg8000 library")
        except ImportError:
            print("  ❌ pg8000 library not available")
            print("  💡 Run the PyPI setup section first to install pg8000")
            return False

        # Create database connection
        conn = pg8000.connect(
            host=POSTGRES_HOST,
            port=POSTGRES_PORT,
            database=POSTGRES_DATABASE,
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD
        )

        print(f"  ✅ Authentication successful")

        # Test basic query
        cursor = conn.cursor()
        cursor.execute("SELECT version();")
        version = cursor.fetchone()[0]
        print(f"  ✅ Database query successful")
        print(f"  📊 PostgreSQL Version: {version[:50]}...")

        # List tables
        cursor.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
            LIMIT 5;
        """)
        tables = cursor.fetchall()
        print(f"  📋 Found {len(tables)} tables in public schema")
        for table in tables[:3]:  # Show first 3 tables
            print(f"     - {table[0]}")

        cursor.close()
        conn.close()
        return True

    except Exception as e:
        print(f"  ❌ Database Error: {e}")
        print(f"  💡 Check your credentials or network connectivity")
        return False

# Run PostgreSQL authentication test
print("=" * 50)
print("POSTGRESQL AUTHENTICATION TEST")
print("=" * 50)

auth_result = test_postgres_authentication()

if auth_result:
    print(f"\n✅ PostgreSQL authentication PASSED")
    print("Your SPCS environment can successfully connect to PostgreSQL!")
    print("You can proceed with PostgreSQL integration.")
else:
    print(f"\n❌ PostgreSQL authentication FAILED")
    print("Check your credentials and ensure network connectivity is working.")
    print("You may also need to configure PostgreSQL EAI (see section below).")


## 3. PostgreSQL EAI Setup (Optional)

Run these cells if connectivity tests failed. This creates the necessary network rules and External Access Integration specifically for PostgreSQL database access.

**Skip this section if your connectivity tests passed.**

In [ ]:
-- Create Network Rule for PostgreSQL connectivity
-- This allows egress traffic to your specific PostgreSQL host and port

-- Network rule for your PostgreSQL database server
-- Uses the variables set in the configuration cell above
CREATE OR REPLACE NETWORK RULE POSTGRES_DB_RULE
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('{{POSTGRES_HOST}}:{{POSTGRES_PORT}}');

-- Show the created network rule
DESCRIBE NETWORK RULE POSTGRES_DB_RULE;

In [ ]:
-- Create External Access Integration for PostgreSQL
-- This combines all the network rules into a single integration

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION POSTGRES_EAI
  ALLOWED_NETWORK_RULES = (
    POSTGRES_DB_RULE
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for PostgreSQL database connectivity';

-- Grant usage on the integration to your role
GRANT USAGE ON INTEGRATION POSTGRES_EAI TO ROLE ACCOUNTADMIN;

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'POSTGRES_EAI';

In [ ]:
-- Apply PostgreSQL EAI to this notebook
-- This enables the notebook to access your PostgreSQL database
-- Include pypi_access_integration if you created it earlier

ALTER NOTEBOOK EAI_POSTGRES
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('POSTGRES_EAI', 'pypi_access_integration');